# Feedback Classification - Baseline Model
## Multilingual (English/Arabic) Text Classification

This notebook implements a baseline model for two-level feedback classification:
- **Level 1**: Technical, Payment, Claims (3 classes)
- **Level 2**: Login, App_Performance, Refund, Limit, Installment, Coverage, Appeal, Preexisting (8 classes)

**Algorithm**: TF-IDF + Logistic Regression

## 1. Import Libraries

In [29]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


## 2. Preprocessing Functions

In [30]:
def preprocess_text(text, lang='en'):
    """
    Preprocess text based on language
    
    Args:
        text: Input text string
        lang: Language code ('en' or 'ar')
    
    Returns:
        Cleaned text string
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtag symbols but keep the text
    text = re.sub(r'#', '', text)
    
    # Remove emojis and special characters (keep Arabic and English letters)
    text = re.sub(r'[^\w\s\u0600-\u06FF]', ' ', text)
    
    # Language-specific preprocessing
    if lang == 'en':
        # Convert to lowercase for English
        text = text.lower()
    
    elif lang == 'ar':
        # Arabic-specific normalization
        # Remove diacritics (تشكيل)
        text = re.sub(r'[ًٌٍَُِّْ]', '', text)
        
        # Normalize different forms of alef
        text = text.replace('أ', 'ا')
        text = text.replace('إ', 'ا')
        text = text.replace('آ', 'ا')
        text = text.replace('ٱ', 'ا')
        
        # Normalize taa marbuta
        text = text.replace('ة', 'ه')
        
        # Normalize yaa
        text = text.replace('ى', 'ي')
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text.strip()


def preprocess_dataframe(df):
    """
    Preprocess entire dataframe
    
    Args:
        df: Input dataframe with 'feedback_text' and 'language' columns
    
    Returns:
        Dataframe with 'cleaned_text' column added
    """
    df = df.copy()
    df['cleaned_text'] = df.apply(
        lambda row: preprocess_text(row['feedback'], row['lang']), 
        axis=1
    )
    # Remove empty texts
    df = df[df['cleaned_text'].str.len() > 0].reset_index(drop=True)
    return df


print("✓ Preprocessing functions defined!")

✓ Preprocessing functions defined!


## 3. Test Preprocessing

In [31]:
# Test English preprocessing
text_en = "Can't log in 😞 #OTP never arrives! @support What's happening?"
cleaned_en = preprocess_text(text_en, 'en')
print(f"Original (EN): {text_en}")
print(f"Cleaned (EN):  {cleaned_en}")
print()

# Test Arabic preprocessing
text_ar = "لا أستطيع تسجيل الدخول، الكود لم يصل! #OTP @الدعم"
cleaned_ar = preprocess_text(text_ar, 'ar')
print(f"Original (AR): {text_ar}")
print(f"Cleaned (AR):  {cleaned_ar}")

Original (EN): Can't log in 😞 #OTP never arrives! @support What's happening?
Cleaned (EN):  can t log in otp never arrives what s happening

Original (AR): لا أستطيع تسجيل الدخول، الكود لم يصل! #OTP @الدعم
Cleaned (AR):  لا استطيع تسجيل الدخول، الكود لم يصل OTP


## 4. Load Training Data

In [32]:
project_path = '/root/AI_Knowledge_Assistant'
data_path = project_path + '/data/raw'
splits_path = data_path + '/splits'
model_path = project_path + '/models'

In [33]:
df_train = pd.read_csv(data_path + '/feedback_train.txt', sep='\t', engine='python', on_bad_lines='warn')


print(f"Training data loaded: {len(df_train)} records")
print(f"\nColumns: {df_train.columns.tolist()}")
print(f"\nFirst few rows:")
df_train.head()

Training data loaded: 412 records

Columns: ['id', 'source_platform', 'lang', 'feedback', 'category_level1', 'category_level2']

First few rows:


,id,source_platform,lang,feedback,category_level1,category_level2
0,1,Twitter,en,Can’t log in 😞 #OTP never arrives! What’s happ...,Technical,Login
1,3,Twitter,ar,لا أستطيع تسجيل الدخول، الكود لم يصل حتى الآن!...,Technical,Login
2,4,Email,ar,أواجه مشكلة في تسجيل الدخول حيث لا يصل رمز الم...,Technical,Login
3,5,Twitter,en,Can’t log in 😞 #OTP never arrives! What’s happ...,Technical,Login
4,7,Twitter,ar,لا أستطيع تسجيل الدخول، الكود لم يصل حتى الآن!...,Technical,Login


## 5. Preprocess Training Data

In [34]:
# Apply preprocessing
df_train = preprocess_dataframe(df_train)

print(f"After preprocessing: {len(df_train)} records")
print(f"\nSample cleaned texts:")
for i in range(min(5, len(df_train))):
    print(f"\n[{df_train.iloc[i]['lang'].upper()}] Original: {df_train.iloc[i]['feedback'][:60]}...")
    print(f"     Cleaned: {df_train.iloc[i]['cleaned_text'][:60]}...")

After preprocessing: 412 records

Sample cleaned texts:

[EN] Original: Can’t log in 😞 #OTP never arrives! What’s happening?...
     Cleaned: can t log in otp never arrives what s happening...

[AR] Original: لا أستطيع تسجيل الدخول، الكود لم يصل حتى الآن! #OTP...
     Cleaned: لا استطيع تسجيل الدخول، الكود لم يصل حتي الان OTP...

[AR] Original: أواجه مشكلة في تسجيل الدخول حيث لا يصل رمز المصادقة إلى هاتف...
     Cleaned: اواجه مشكله في تسجيل الدخول حيث لا يصل رمز المصادقه الي هاتف...

[EN] Original: Can’t log in 😞 #OTP never arrives! What’s happening?...
     Cleaned: can t log in otp never arrives what s happening...

[AR] Original: لا أستطيع تسجيل الدخول، الكود لم يصل حتى الآن! #OTP...
     Cleaned: لا استطيع تسجيل الدخول، الكود لم يصل حتي الان OTP...


## 6. Prepare Features and Labels

In [35]:
# Extract features and labels
X = df_train['cleaned_text'].values
y_level1 = df_train['category_level1'].values
y_level2 = df_train['category_level2'].values

print(f"Features (X): {len(X)} samples")
print(f"Labels Level 1: {len(np.unique(y_level1))} classes - {np.unique(y_level1)}")
print(f"Labels Level 2: {len(np.unique(y_level2))} classes - {np.unique(y_level2)}")

Features (X): 412 samples
Labels Level 1: 3 classes - ['Claims' 'Payment' 'Technical']
Labels Level 2: 8 classes - ['App_Performance' 'Appeal' 'Coverage' 'Installment' 'Limit' 'Login'
 'Preexisting' 'Refund']


## 7. Vectorization (TF-IDF)

In [36]:
# Initialize TF-IDF vectorizers
vectorizer_level1 = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),  # unigrams + bigrams
    min_df=2,
    sublinear_tf=True
)

vectorizer_level2 = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2,
    sublinear_tf=True
)

print("✓ TF-IDF vectorizers initialized!")

✓ TF-IDF vectorizers initialized!


## 8.1 Training - Option A: Train/Validation Split

In [37]:
# # Split data (80% train, 20% validation)
# X_train, X_val, y_train_l1, y_val_l1, y_train_l2, y_val_l2 = train_test_split(
#     X, y_level1, y_level2,
#     test_size=0.2,
#     random_state=42,
#     stratify=y_level1  # Maintain class distribution
# )

# print(f"Training samples: {len(X_train)}")
# print(f"Validation samples: {len(X_val)}")

# # Vectorize
# X_train_tfidf_l1 = vectorizer_level1.fit_transform(X_train)
# X_val_tfidf_l1 = vectorizer_level1.transform(X_val)

# X_train_tfidf_l2 = vectorizer_level2.fit_transform(X_train)
# X_val_tfidf_l2 = vectorizer_level2.transform(X_val)

# print(f"\nTF-IDF features Level 1: {X_train_tfidf_l1.shape[1]}")
# print(f"TF-IDF features Level 2: {X_train_tfidf_l2.shape[1]}")

# # Train Level 1 Model
# print("\n" + "="*60)
# print("TRAINING LEVEL 1 CLASSIFIER (3 classes)")
# print("="*60)

# model_level1 = LogisticRegression(
#     max_iter=1000,
#     random_state=42,
#     class_weight='balanced',  # Handle class imbalance
#     C=1.0
# )

# model_level1.fit(X_train_tfidf_l1, y_train_l1)

# # Predict and evaluate
# y_pred_l1 = model_level1.predict(X_val_tfidf_l1)
# acc_l1 = accuracy_score(y_val_l1, y_pred_l1)
# f1_l1 = f1_score(y_val_l1, y_pred_l1, average='weighted')

# print(f"\nValidation Accuracy: {acc_l1:.4f}")
# print(f"Validation F1-score: {f1_l1:.4f}")
# print("\nClassification Report:")
# print(classification_report(y_val_l1, y_pred_l1))

# # Train Level 2 Model
# print("\n" + "="*60)
# print("TRAINING LEVEL 2 CLASSIFIER (8 classes)")
# print("="*60)

# model_level2 = LogisticRegression(
#     max_iter=1000,
#     random_state=42,
#     class_weight='balanced',
#     C=1.0
# )

# model_level2.fit(X_train_tfidf_l2, y_train_l2)

# # Predict and evaluate
# y_pred_l2 = model_level2.predict(X_val_tfidf_l2)
# acc_l2 = accuracy_score(y_val_l2, y_pred_l2)
# f1_l2 = f1_score(y_val_l2, y_pred_l2, average='weighted')

# print(f"\nValidation Accuracy: {acc_l2:.4f}")
# print(f"Validation F1-score: {f1_l2:.4f}")
# print("\nClassification Report:")
# print(classification_report(y_val_l2, y_pred_l2))

# print("\n" + "="*60)
# print("TRAINING COMPLETED!")
# print("="*60)

## 8.2 Training - Cross-Validation (Recommended for Small Datasets)

In [38]:
print("="*60)
print("CROSS-VALIDATION (5 FOLDS)")
print("="*60)

# Vectorize full dataset
X_tfidf_l1 = vectorizer_level1.fit_transform(X)
X_tfidf_l2 = vectorizer_level2.fit_transform(X)

# Train Level 1 Model
print("\n" + "="*60)
print("TRAINING LEVEL 1 CLASSIFIER (3 classes)")
print("="*60)

model_level1 = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced',  # Handle class imbalance
    C=1.0
)

# Train Level 2 Model
print("\n" + "="*60)
print("TRAINING LEVEL 2 CLASSIFIER (8 classes)")
print("="*60)

model_level2 = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced',
    C=1.0
)

# Level 1 Cross-Validation
print("\n--- Level 1 Cross-Validation ---")
cv_scores_l1 = cross_val_score(
    model_level1, X_tfidf_l1, y_level1,
    cv=5, scoring='f1_weighted'
)
print(f"CV F1-scores: {cv_scores_l1}")
print(f"Mean F1: {cv_scores_l1.mean():.4f} (+/- {cv_scores_l1.std() * 2:.4f})")

# Level 2 Cross-Validation
print("\n--- Level 2 Cross-Validation ---")
cv_scores_l2 = cross_val_score(
    model_level2, X_tfidf_l2, y_level2,
    cv=5, scoring='f1_weighted'
)
print(f"CV F1-scores: {cv_scores_l2}")
print(f"Mean F1: {cv_scores_l2.mean():.4f} (+/- {cv_scores_l2.std() * 2:.4f})")

# Train on full dataset for final models
print("\n--- Training on Full Dataset ---")
model_level1.fit(X_tfidf_l1, y_level1)
model_level2.fit(X_tfidf_l2, y_level2)
print("✓ Final models trained on full dataset")

CROSS-VALIDATION (5 FOLDS)

TRAINING LEVEL 1 CLASSIFIER (3 classes)

TRAINING LEVEL 2 CLASSIFIER (8 classes)

--- Level 1 Cross-Validation ---
CV F1-scores: [1. 1. 1. 1. 1.]
Mean F1: 1.0000 (+/- 0.0000)

--- Level 2 Cross-Validation ---
CV F1-scores: [1. 1. 1. 1. 1.]
Mean F1: 1.0000 (+/- 0.0000)

--- Training on Full Dataset ---
✓ Final models trained on full dataset


## 9. Save Trained Models

In [39]:
# Save models and vectorizers
joblib.dump(vectorizer_level1, model_path + '/vectorizer_level1.pkl')
joblib.dump(vectorizer_level2, model_path + '/vectorizer_level2.pkl')
joblib.dump(model_level1, model_path + '/model_level1.pkl')
joblib.dump(model_level2, model_path + '/model_level2.pkl')

print("✓ Models saved successfully!")
print("  - vectorizer_level1.pkl")
print("  - vectorizer_level2.pkl")
print("  - model_level1.pkl")
print("  - model_level2.pkl")

✓ Models saved successfully!
  - vectorizer_level1.pkl
  - vectorizer_level2.pkl
  - model_level1.pkl
  - model_level2.pkl


## 10. Load Evaluation Data

In [40]:
df_eval = pd.read_csv(data_path + '/feedback_eval.txt', sep='\t', engine='python', on_bad_lines='warn')


print(f"Evaluation data loaded: {len(df_eval)} records")
df_eval

Evaluation data loaded: 200 records


,id,source_platform,lang,feedback
0,1,Twitter,en,Can’t log in 😞 #OTP never arrives! What’s happ...
1,2,Email,en,Hello
2,3,Twitter,ar,لا أستطيع تسجيل الدخول، الكود لم يصل حتى الآن!...
3,4,Email,ar,أواجه مشكلة في تسجيل الدخول حيث لا يصل رمز الم...
4,5,Twitter,en,Can’t log in 😞 #OTP never arrives! What’s happ...
...,...,...,...,...
195,196,Twitter,en,Can’t log in 😞 #OTP never arrives! What’s happ...
196,197,Twitter,en,Can’t log in 😞 #OTP never arrives! What’s happ...
197,198,Google Maps,ar,التطبيق بطيء جدًا ويتوقف عند تحميل المستندات!
198,199,LinkedIn,en,I have a chronic condition. How does that affe...


## 11. Preprocess Evaluation Data

In [41]:
# Preprocess evaluation data
df_eval = preprocess_dataframe(df_eval)

print(f"After preprocessing: {len(df_eval)} records")
print("\nCleaned texts:")
for i in range(len(df_eval)):
    print(f"{i+1}. [{df_eval.iloc[i]['lang'].upper()}] {df_eval.iloc[i]['cleaned_text']}")

After preprocessing: 200 records

Cleaned texts:
1. [EN] can t log in otp never arrives what s happening
2. [EN] hello
3. [AR] لا استطيع تسجيل الدخول، الكود لم يصل حتي الان OTP
4. [AR] اواجه مشكله في تسجيل الدخول حيث لا يصل رمز المصادقه الي هاتفي
5. [EN] can t log in otp never arrives what s happening
6. [EN] hello
7. [AR] لا استطيع تسجيل الدخول، الكود لم يصل حتي الان OTP
8. [AR] اواجه مشكله في تسجيل الدخول حيث لا يصل رمز المصادقه الي هاتفي
9. [EN] can t log in otp never arrives what s happening
10. [EN] hello
11. [AR] لا استطيع تسجيل الدخول، الكود لم يصل حتي الان OTP
12. [AR] اواجه مشكله في تسجيل الدخول حيث لا يصل رمز المصادقه الي هاتفي
13. [EN] can t log in otp never arrives what s happening
14. [EN] hello
15. [AR] لا استطيع تسجيل الدخول، الكود لم يصل حتي الان OTP
16. [AR] اواجه مشكله في تسجيل الدخول حيث لا يصل رمز المصادقه الي هاتفي
17. [EN] can t log in otp never arrives what s happening
18. [EN] hello
19. [AR] لا استطيع تسجيل الدخول، الكود لم يصل حتي الان OTP
20. [EN] app keeps cr

## 12. Make Predictions

In [42]:
# Extract text
X_eval = df_eval['cleaned_text'].values

# Vectorize
X_eval_tfidf_l1 = vectorizer_level1.transform(X_eval)
X_eval_tfidf_l2 = vectorizer_level2.transform(X_eval)

# Predict Level 1
pred_level1 = model_level1.predict(X_eval_tfidf_l1)
proba_level1 = model_level1.predict_proba(X_eval_tfidf_l1).max(axis=1)

# Predict Level 2
pred_level2 = model_level2.predict(X_eval_tfidf_l2)
proba_level2 = model_level2.predict_proba(X_eval_tfidf_l2).max(axis=1)

print("✓ Predictions completed!")

✓ Predictions completed!


## 13. Create Output DataFrame

In [43]:
# Create predictions dataframe
df_predictions = pd.DataFrame({
    'record_id': df_eval['id'],
    'predicted_level1': pred_level1,
    'probability_level1': proba_level1.round(4),
    'predicted_level2': pred_level2,
    'probability_level2': proba_level2.round(4)
})

print("\n" + "="*60)
print("PREDICTIONS")
print("="*60)
df_predictions


PREDICTIONS


,record_id,predicted_level1,probability_level1,predicted_level2,probability_level2
0,1,Technical,0.8648,Login,0.7782
1,2,Claims,0.4002,App_Performance,0.1373
2,3,Technical,0.9136,Login,0.8599
3,4,Technical,0.9032,Login,0.8379
4,5,Technical,0.8648,Login,0.7782
...,...,...,...,...,...
195,196,Technical,0.8648,Login,0.7782
196,197,Technical,0.8648,Login,0.7782
197,198,Technical,0.8981,App_Performance,0.7991
198,199,Claims,0.8696,Preexisting,0.7735


## 14. Display Predictions with Original Text

In [44]:
# Create detailed view
df_detailed = df_eval[['id', 'feedback', 'lang']].copy()
df_detailed['predicted_level1'] = pred_level1
df_detailed['prob_l1'] = proba_level1.round(4)
df_detailed['predicted_level2'] = pred_level2
df_detailed['prob_l2'] = proba_level2.round(4)

print("\n" + "="*60)
print("DETAILED PREDICTIONS")
print("="*60)

for i in range(len(df_detailed)):
    row = df_detailed.iloc[i]
    print(f"\n{i+1}. ID: {row['id']}")
    print(f"   Text: {row['feedback']}")
    print(f"   Language: {row['lang']}")
    print(f"   Level 1: {row['predicted_level1']} (confidence: {row['prob_l1']:.2%})")
    print(f"   Level 2: {row['predicted_level2']} (confidence: {row['prob_l2']:.2%})")


DETAILED PREDICTIONS

1. ID: 1
   Text: Can’t log in 😞 #OTP never arrives! What’s happening?
   Language: en
   Level 1: Technical (confidence: 86.48%)
   Level 2: Login (confidence: 77.82%)

2. ID: 2
   Text: Hello
   Language: en
   Level 1: Claims (confidence: 40.02%)
   Level 2: App_Performance (confidence: 13.73%)

3. ID: 3
   Text: لا أستطيع تسجيل الدخول، الكود لم يصل حتى الآن! #OTP
   Language: ar
   Level 1: Technical (confidence: 91.36%)
   Level 2: Login (confidence: 85.99%)

4. ID: 4
   Text: أواجه مشكلة في تسجيل الدخول حيث لا يصل رمز المصادقة إلى هاتفي.
   Language: ar
   Level 1: Technical (confidence: 90.32%)
   Level 2: Login (confidence: 83.79%)

5. ID: 5
   Text: Can’t log in 😞 #OTP never arrives! What’s happening?
   Language: en
   Level 1: Technical (confidence: 86.48%)
   Level 2: Login (confidence: 77.82%)

6. ID: 6
   Text: Hello
   Language: en
   Level 1: Claims (confidence: 40.02%)
   Level 2: App_Performance (confidence: 13.73%)

7. ID: 7
   Text: لا أستطيع 

## 15. Save Predictions to Files

In [45]:
predictions_path = '/root/AI_Knowledge_Assistant/data/predictions'

In [46]:
# Save to CSV
df_predictions.to_csv(predictions_path + '/feedback_predictions.csv', index=False)
print("✓ Predictions saved to: feedback_predictions.csv")


# Display file contents
print("\nCSV Preview:")
print(df_predictions.to_csv(index=False))

✓ Predictions saved to: feedback_predictions.csv

CSV Preview:
record_id,predicted_level1,probability_level1,predicted_level2,probability_level2
1,Technical,0.8648,Login,0.7782
2,Claims,0.4002,App_Performance,0.1373
3,Technical,0.9136,Login,0.8599
4,Technical,0.9032,Login,0.8379
5,Technical,0.8648,Login,0.7782
6,Claims,0.4002,App_Performance,0.1373
7,Technical,0.9136,Login,0.8599
8,Technical,0.9032,Login,0.8379
9,Technical,0.8648,Login,0.7782
10,Claims,0.4002,App_Performance,0.1373
11,Technical,0.9136,Login,0.8599
12,Technical,0.9032,Login,0.8379
13,Technical,0.8648,Login,0.7782
14,Claims,0.4002,App_Performance,0.1373
15,Technical,0.9136,Login,0.8599
16,Technical,0.9032,Login,0.8379
17,Technical,0.8648,Login,0.7782
18,Claims,0.4002,App_Performance,0.1373
19,Technical,0.9136,Login,0.8599
20,Technical,0.8845,App_Performance,0.7757
21,Technical,0.8957,App_Performance,0.7974
22,Technical,0.8981,App_Performance,0.7991
23,Technical,0.8986,App_Performance,0.7972
24,Technical,0.8845,App_Perfor

## 16. Prediction Analysis & Statistics

In [47]:
print("="*60)
print("PREDICTION STATISTICS")
print("="*60)

print("\nLevel 1 Predictions:")
print(df_predictions['predicted_level1'].value_counts())

print("\nLevel 2 Predictions:")
print(df_predictions['predicted_level2'].value_counts())

print("\nConfidence Statistics:")
print(f"Level 1 - Mean: {df_predictions['probability_level1'].mean():.4f}, "
      f"Min: {df_predictions['probability_level1'].min():.4f}, "
      f"Max: {df_predictions['probability_level1'].max():.4f}")
print(f"Level 2 - Mean: {df_predictions['probability_level2'].mean():.4f}, "
      f"Min: {df_predictions['probability_level2'].min():.4f}, "
      f"Max: {df_predictions['probability_level2'].max():.4f}")

# Low confidence predictions
threshold = 0.5
low_conf_l1 = df_predictions[df_predictions['probability_level1'] < threshold]
low_conf_l2 = df_predictions[df_predictions['probability_level2'] < threshold]

print(f"\nLow Confidence Predictions (< {threshold}):")
print(f"Level 1: {len(low_conf_l1)} predictions")
print(f"Level 2: {len(low_conf_l2)} predictions")

PREDICTION STATISTICS

Level 1 Predictions:
predicted_level1
Payment      78
Claims       75
Technical    47
Name: count, dtype: int64

Level 2 Predictions:
predicted_level2
App_Performance    34
Limit              29
Installment        25
Refund             24
Preexisting        23
Coverage           23
Appeal             22
Login              20
Name: count, dtype: int64

Confidence Statistics:
Level 1 - Mean: 0.8474, Min: 0.4002, Max: 0.9136
Level 2 - Mean: 0.7500, Min: 0.1373, Max: 0.8599

Low Confidence Predictions (< 0.5):
Level 1: 7 predictions
Level 2: 14 predictions


## 17. Load Saved Models (For Future Use)

In [48]:
# If you want to load previously saved models:

# vectorizer_level1 = joblib.load('vectorizer_level1.pkl')
# vectorizer_level2 = joblib.load('vectorizer_level2.pkl')
# model_level1 = joblib.load('model_level1.pkl')
# model_level2 = joblib.load('model_level2.pkl')

# print("✓ Models loaded successfully!")

## 18. Quick Prediction Function

In [49]:
def predict_single_feedback(text, language):
    """
    Predict category for a single feedback text
    
    Args:
        text: Feedback text
        language: 'en' or 'ar'
    
    Returns:
        Dictionary with predictions and probabilities
    """
    # Preprocess
    cleaned = preprocess_text(text, language)
    
    # Vectorize
    X_vec_l1 = vectorizer_level1.transform([cleaned])
    X_vec_l2 = vectorizer_level2.transform([cleaned])
    
    # Predict
    pred_l1 = model_level1.predict(X_vec_l1)[0]
    prob_l1 = model_level1.predict_proba(X_vec_l1).max()
    
    pred_l2 = model_level2.predict(X_vec_l2)[0]
    prob_l2 = model_level2.predict_proba(X_vec_l2).max()
    
    return {
        'text': text,
        'language': language,
        'level1': pred_l1,
        'level1_confidence': f"{prob_l1:.2%}",
        'level2': pred_l2,
        'level2_confidence': f"{prob_l2:.2%}"
    }


# Test the function
test_text_en = "I can't access my account!"
result_en = predict_single_feedback(test_text_en, 'en')
print("English Test:")
print(result_en)

test_text_ar = "متى سأستلم المبلغ المسترد؟"
result_ar = predict_single_feedback(test_text_ar, 'ar')
print("\nArabic Test:")
print(result_ar)

English Test:
{'text': "I can't access my account!", 'language': 'en', 'level1': 'Payment', 'level1_confidence': '49.99%', 'level2': 'Limit', 'level2_confidence': '28.72%'}

Arabic Test:
{'text': 'متى سأستلم المبلغ المسترد؟', 'language': 'ar', 'level1': 'Payment', 'level1_confidence': '77.46%', 'level2': 'Refund', 'level2_confidence': '59.69%'}


## 19. Experiment: Compare Different Hyperparameters

In [50]:
# Experiment with different TF-IDF configurations

configs = [
    {'name': 'Unigrams only', 'max_features': 3000, 'ngram_range': (1, 1)},
    {'name': 'Unigrams + Bigrams', 'max_features': 5000, 'ngram_range': (1, 2)},
    {'name': 'Up to Trigrams', 'max_features': 10000, 'ngram_range': (1, 3)},
]

results = []

for config in configs:
    print(f"\nTesting: {config['name']}")
    print("-" * 40)
    
    # Create vectorizer
    vec = TfidfVectorizer(
        max_features=config['max_features'],
        ngram_range=config['ngram_range'],
        min_df=2
    )
    
    # Vectorize
    X_vec = vec.fit_transform(X)
    
    # Train and evaluate with CV
    clf = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
    
    scores_l1 = cross_val_score(clf, X_vec, y_level1, cv=3, scoring='f1_weighted')
    scores_l2 = cross_val_score(clf, X_vec, y_level2, cv=3, scoring='f1_weighted')
    
    print(f"Level 1 F1: {scores_l1.mean():.4f}")
    print(f"Level 2 F1: {scores_l2.mean():.4f}")
    
    results.append({
        'config': config['name'],
        'max_features': config['max_features'],
        'ngram_range': str(config['ngram_range']),
        'level1_f1': scores_l1.mean(),
        'level2_f1': scores_l2.mean()
    })

# Display comparison
df_results = pd.DataFrame(results)
print("\n" + "="*60)
print("HYPERPARAMETER COMPARISON")
print("="*60)
df_results


Testing: Unigrams only
----------------------------------------
Level 1 F1: 0.9745
Level 2 F1: 1.0000

Testing: Unigrams + Bigrams
----------------------------------------
Level 1 F1: 0.9745
Level 2 F1: 1.0000

Testing: Up to Trigrams
----------------------------------------
Level 1 F1: 0.9746
Level 2 F1: 1.0000

HYPERPARAMETER COMPARISON


,config,max_features,ngram_range,level1_f1,level2_f1
0,Unigrams only,3000,"(1, 1)",0.974546,1.0
1,Unigrams + Bigrams,5000,"(1, 2)",0.974546,1.0
2,Up to Trigrams,10000,"(1, 3)",0.974564,1.0


## 20. Summary & Next Steps

In [51]:
print("="*60)
print("BASELINE MODEL SUMMARY")
print("="*60)

print("\n✓ Completed Tasks:")
print("  1. Text preprocessing (English & Arabic)")
print("  2. TF-IDF vectorization")
print("  3. Two-level classification models trained")
print("  4. Model evaluation with metrics")
print("  5. Predictions on evaluation data")
print("  6. Output files created (CSV & JSON)")

print("\n📊 Model Performance:")
print(f"  Level 1 (3 classes): Accuracy = {acc_l1:.2%}, F1 = {f1_l1:.2%}")
print(f"  Level 2 (8 classes): Accuracy = {acc_l2:.2%}, F1 = {f1_l2:.2%}")

print("\n📁 Output Files:")
print("  - feedback_predictions.csv")
print("  - feedback_predictions.json")
print("  - vectorizer_level1.pkl")
print("  - vectorizer_level2.pkl")
print("  - model_level1.pkl")
print("  - model_level2.pkl")

print("\n🚀 Next Steps:")
print("  1. Collect more training data for better performance")
print("  2. Try deep learning models (mBERT, XLM-RoBERTa)")
print("  3. Implement API endpoint for production")
print("  4. Add confidence-based filtering")
print("  5. Monitor and retrain with new data")

print("\n" + "="*60)

BASELINE MODEL SUMMARY

✓ Completed Tasks:
  1. Text preprocessing (English & Arabic)
  2. TF-IDF vectorization
  3. Two-level classification models trained
  4. Model evaluation with metrics
  5. Predictions on evaluation data
  6. Output files created (CSV & JSON)

📊 Model Performance:


NameError: name 'acc_l1' is not defined